In [3]:
import numpy as np

In [4]:
# Parameters
S0 = 100  # Initial asset price
K = 100  # Strike price
T = 1.0  # Time to maturity in years
r = 0.05  # Risk-free rate
sigma = 0.2  # Volatility
B = 70  # Barrier level
C = 0.1  # Coupon rate
N = 252  # Number of time steps (daily steps)
M = 10000  # Number of simulations
autocall_levels = [105, 110, 115]  # Example auto-call levels at observation dates
autocall_dates = [0.25, 0.5, 0.75]  # Observation dates (in years)

dt = T / N  # Time step size

# Monte Carlo simulation
payoffs = np.zeros(M)

for i in range(M):
    S = np.zeros(N + 1)
    S[0] = S0
    is_autocalled = False
    for t in range(1, N + 1):
        Z = np.random.standard_normal()
        S[t] = S[t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
        
        # Check for auto-call condition
        if t * dt in autocall_dates:
            index = autocall_dates.index(t * dt)
            if S[t] >= autocall_levels[index]:
                payoffs[i] = S[t] * C * (index + 1)
                is_autocalled = True
                break
        
        # Check for barrier condition
        if S[t] <= B:
            payoffs[i] = S[t] * C  # Modify as needed for specific payoff structure
            is_autocalled = True
            break
    
    if not is_autocalled:
        if S[-1] >= K:
            payoffs[i] = S[-1] * (1 + C)  # Modify as needed for specific payoff structure
        else:
            payoffs[i] = S[-1] * (1 - C)  # Modify as needed for specific payoff structure

# Discount payoffs to present value
payoffs_discounted = np.exp(-r * T) * payoffs

# Calculate the average discounted payoff
price = np.mean(payoffs_discounted)

print(f"Estimated price of the ABRC derivative: {price:.2f}")


Estimated price of the ABRC derivative: 51.15
